## How to use the StoG class from PyStoG



In [ ]:
%matplotlib inline

Now, lets load in some test data for Argon from our test suite. We extract the $Q$ and $S(Q)$ 
along with the $r$ and $g(r) as well as some material-specific information to initialize StoG.

In [ ]:
cat ../data/test_data/argon.reciprocal_space.dat

In [ ]:
cat ../data/test_data/argon.real_space.dat

In [ ]:
import numpy as np
from tests.materials import Argon
from tests.utils import get_index_of_function
from pystog.utils import ReciprocalSpaceHeaders, RealSpaceHeaders

material = Argon()
    
reciprocal_space_filename = "../data/test_data/argon.reciprocal_space.dat"

data = np.loadtxt(reciprocal_space_filename, skiprows=2)
q = data[:, get_index_of_function("Q", ReciprocalSpaceHeaders)]
sq = data[:, get_index_of_function("S(Q)", RealSpaceHeaders)]

real_space_filename = "../data/test_data/argon.real_space.dat"
                 
data = np.loadtxt(real_space_filename, skiprows=2)
r = data[:, get_index_of_function("r", RealSpaceHeaders)]
gofr = data[:, get_index_of_function("g(r)", RealSpaceHeaders)]

kwargs_for_stog_input = {
    'NumberDensity': material.kwargs['rho'],
    '<b_coh>^2': material.kwargs['<b_coh>^2'],
    '<b_tot^2>': material.kwargs['<b_tot^2>'],
    'FourierFilter': {'Cutoff': 1.5},
    'OmittedXrangeCorrection': False,
    'Rdelta': r[1] -r[0],
    'Rmin': min(r),
    'Rmax': max(r)
}

kwargs_for_files = {
    'Files': [
        {'Filename': reciprocal_space_filename,
         'ReciprocalFunction': 'S(Q)',
         'Qmin': 0.02,
         'Qmax': 15.0,
         'Y': {'Offset': 0.0,
               'Scale': 1.0},
         'X': {'Offset': 0.0}
         },
        {'Filename': reciprocal_space_filename,
         'ReciprocalFunction': 'S(Q)',
         'Qmin': 1.90,
         'Qmax': 35.2,
         'Y': {'Offset': 0.0,
               'Scale': 1.0},
         'X': {'Offset': 0.0}
         }
    ]
}


In [ ]:
from pystog import StoG
stog = StoG(**kwargs_for_stog_input)
stog.plot_flag = False
stog.stem_name = "dog"
stog.files = kwargs_for_files['Files']
stog.read_all_data()
stog.merge_data()
stog.transform_merged()
stog.fourier_filter()

In [ ]:
stog.plot_merged_sq()

In [ ]:
stog.plot_sq()

In [ ]:
stog.plot_gr()